In [1]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_enrich_recipes.json') as f:
    gpt4_recipe = json.load(f)

In [2]:
import pandas as pd

/var/folders/qb/cr87d2s56q51lk72620820hw0000gn/T/ipykernel_60461/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('../data/generated_recipe_v2/ingredients.csv')

In [4]:
df

,name,portion,weight(g)
0,cucumber,"1 slice, 1 stick, 1 medium, 1 small, 1 cup","10, 10, 200, 65, 120"
1,carrots,"1 cup, 1 regular, 1 medium","120, 60, 60"
2,olive oil,"1 cup, 1 tablespoon","224,14"
3,lemon,"1 slice, 1 wedge, 1 lemon","8,8,65"
4,thyme,"1 tablespoon, 1 teaspoon","4.3, 1.4"
5,spinach,"1 cup, 1 leaf, 1 babyleaf","25, 10, 0.6"
6,garlic,"1 cup, 1 clove, 1 teaspoon","135, 3, 5"
7,feta cheese,"1 wedge, 1 cup, 1 cubic inch","38, 150, 17"
8,milk,"1 cup, 1 fl oz","244, 30.5"
9,almond milk,"1 cup, 1 fl oz","244, 30.5"


In [5]:
df[df['name'].str.contains('chia seeds')]

,name,portion,weight(g)
30,chia seeds,"1 cup, 1 tablespoon","168, 10.5"


In [6]:
FRACTIONS = {
    "½": 0.5,
    "⅓": 1 / 3,
    "⅔": 2 / 3,
    "¼": 0.25,
    "¾": 0.75,
    "⅕": 0.2,
    "⅖": 0.4,
    "⅗": 0.6,
    "⅘": 0.8,
    "⅙": 1 / 6,
    "⅚": 5 / 6,
    "⅛": 0.125,
    "⅜": 0.375,
    "⅝": 0.625,
    "⅞": 0.875,
}


In [7]:
def _extract_fractional(input_string: str) -> float:
    input_string = input_string.strip()

    # Handling mixed numbers with unicode fractions e.g., '1⅔'
    for unicode_fraction, fraction_part in FRACTIONS.items():
        if unicode_fraction in input_string:
            whole_number_part, _, _ = input_string.partition(unicode_fraction)

            whole_number = float(whole_number_part or 0)
            return whole_number + fraction_part

    if input_string in FRACTIONS:
        return FRACTIONS[input_string]

    try:
        return round(float(input_string),2)
    except ValueError:
        pass

    if " " in input_string and "/" in input_string:
        whole_part, fractional_part = input_string.split(" ", 1)
        numerator, denominator = fractional_part.split("/")
        return round(float(whole_part) + float(numerator) / float(denominator), 2)

    elif "/" in input_string:
        numerator, denominator = input_string.split("/")
        return round(float(numerator) / float(denominator), 2)

    raise ValueError(f"Unrecognized fraction format: '{input_string}'")

In [8]:
import sys
sys.path.append('/Users/divyadhara/Documents/CPSC597/ai-meal-recommendation')
from utils.models import EnrichRecipesWithEnergy

In [9]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_enrich_recipes.json') as f:
    gpt4_enrich_recipe = json.load(f)

In [10]:
recipes=EnrichRecipesWithEnergy(**gpt4_enrich_recipe)

In [11]:
recipes

EnrichRecipesWithEnergy(recipes=[EnrichRecipeWithEnergy(id='14', recipe_name='Mediterranean Chickpea Salad', ingredients=[IngredientWithEngergy(name='chickpeas (cooked or canned)', quantity='2 cups', ner_name='chickpeas', usda_food_ingredient=USDAFoodIngredient(fdc_id=2644282, name='"chickpeas"', nutrition_id=2047, energy=383.0, unit='KCAL')), IngredientWithEngergy(name='cucumber (diced)', quantity='1 medium', ner_name='cucumber', usda_food_ingredient=USDAFoodIngredient(fdc_id=2346406, name='"cucumber"', nutrition_id=2047, energy=15.9, unit='KCAL')), IngredientWithEngergy(name='cherry tomatoes (halved)', quantity='1 cup', ner_name='cherry tomatoes', usda_food_ingredient=USDAFoodIngredient(fdc_id=2394234, name='"cherry tomatoes"', nutrition_id=1008, energy=433.0, unit='KCAL')), IngredientWithEngergy(name='red onion (finely chopped)', quantity='1/4 cup', ner_name='red onion', usda_food_ingredient=USDAFoodIngredient(fdc_id=790577, name='"onion"', nutrition_id=1008, energy=44.0, unit='KCAL

In [12]:
import re

def extract_quantity_and_unit(text):
    # Using a regex pattern to match number followed by one of the specified units
    pattern = r"(\d+\.?\d*)\s*(ounces|ounce|grams|gram|oz|g)"
    match = re.search(pattern, text)
    if match:
        quantity = match.group(1)  # The numerical quantity
        unit = match.group(2)      # The unit (e.g., ounces, grams, oz, g)
        return quantity, unit
    else:
        return "NA",'NA'

# Example usage
text = "15 cup"
quantity, unit = extract_quantity_and_unit(text)
print(f"Extracted Quantity: {quantity} {unit}")

Extracted Quantity: NA NA


In [13]:
 RECIPE_YIELD_TYPES = (
    ("dozen", "dozen"),
    ("batch", "batches"),
    ("cake", "cakes"),
    ("sandwich", "sandwiches"),
    ("bun", "buns"),
    ("cookie", "cookies"),
    ("muffin", "muffins"),
    ("cupcake", "cupcakes"),
    ("loaf", "loaves"),
    ("pie", "pies"),
    ("cup", "cups"),
    ("pint", "pints"),
    ("gallon", "gallons"),
    ("ounce", "ounces"),
    ("pound", "pounds"),
    ("gram", "grams"),
    ("liter", "liters"),
    ("piece", "pieces"),
    ("layer", "layers"),
    ("scoop", "scoops"),
    ("bar", "bars"),
    ("patty", "patties"),
    ("hamburger bun", "hamburger buns"),
    ("pancake", "pancakes"),
    ("item", "items"),
    # ... add more types as needed, in (singular, plural) format ...
)

In [14]:
import re

def clean_string(input_string):
    # Convert to lowercase
    input_string = input_string.lower()

    # Define replacement rules
    replacements = {
        r'\bfresh\b': '',
        r'\bgrated\b': '',
        r'\bextra virgin\b': '',
        r'\bextravirgin\b': '',
        r'\bvirgin\b': '',
        r'\blowfat\b': '',
        r'\bred\b': '',
        r'\bgreen\b': '',
        r'\bwhite\b': '',
        r'\bwhites\b': '',
        r'\bbrown\b': '',
        r'\bblack\b': '',
        r'\bkalamata\b': '',
        r'\bstalks\b': '',
        r'\bleaves\b': '',
        r'\bcremini\b': '',
        r'\bslices\b': '',
        r'\bzest\b': '',
        r'\bzucchinis\b': 'zucchini'
    }

    # Apply replacements using regular expressions
    for pattern, replacement in replacements.items():
        input_string = re.sub(pattern, replacement, input_string)

    # Remove extra spaces by splitting and joining words
    return ' '.join(input_string.split())



In [15]:
clean_string('lemon zest')

'lemon'

In [16]:
def clean_hyphen(input_string):
    if '-' in input_string:
        parts = input_string.split("-")
        return parts[0]
    return input_string

In [17]:
def parse_quantity(quantity: str):
    quantity = quantity.replace('packed','').strip()
    quantity = quantity.replace('pack','').strip()
    quantity = quantity.replace('(optional)','').strip()
    quantity = ' '.join(quantity.split())
    if 'taste' in quantity:
        return 1.0, 'tablespoon'
    portion, unit = extract_quantity_and_unit(quantity)
    if portion != 'NA' and unit != 'NA':
        return portion, unit 
    parts = quantity.split(" ")
    if len(parts) == 0:
        return 'NA', 'NA'
    if len(parts) == 1:
        try:
            clean_input = clean_hyphen(parts[0])
            result = _extract_fractional(clean_input)
            if isinstance(result, float):
                return result,'NA'
        except ValueError as e:
            return 1.0, parts[0]
    if len(parts) == 2:
        clean_input = clean_hyphen(parts[0])
        return _extract_fractional(clean_input),parts[1]
    else:
        return 'NA', 'NA'


In [18]:
extract_quantity_and_unit('1')

('NA', 'NA')

In [19]:
parse_quantity('1 cup (optional)')

(1.0, 'cup')

In [20]:
from pydantic import BaseModel
from typing import  Optional
class IngredientUnitPortion(BaseModel):
    name: str
    clean_name: str
    quantity: str
    portion: str|float
    unit: str
    weight: float|str

In [21]:
from typing import  List
class IngredientEnrich(BaseModel):
    recipe_id: int
    ingredient_unit_portion:List[IngredientUnitPortion]=[]

class IngredientsEnrich(BaseModel):
    ingredients:List[IngredientEnrich]=[]

In [22]:
PLURAL_TO_SINGULAR={
    'cups': 'cup',
    'slices': 'slice',
    'pieces': 'piece',
    'teaspoons': 'teaspoon',
    'tablespoons': 'tablespoon',
    'cloves': 'clove',
    'ounces': 'ounce',
    'ozs': 'oz'
}

In [23]:
df['name'] = df['name'].str.lower()

In [24]:
def index_contaning_substring(portions:List[str], input_string:str):
    for index, item in enumerate(portions):
        parts = item.split(' ')
        if parts[1].strip() in input_string and parts[1].strip() in item:
            return index
    return -1

Conversion to weight for ingredient using portion and unit

In [25]:
def query_df(df:pd.DataFrame,query:str, portion:float, unit:str):
    if 'ounce' in unit:
        return float(portion) * 28.0
    rows = df[df['name'].str.contains(query)]
    items_found = len(rows)
    retry = 0
    for index, row in rows.iterrows():
        portions = [item.strip() for item in row['portion'].split(',')]
        index = index_contaning_substring(portions,unit)
        if index == -1:
            continue
        if retry == items_found:
            index=0
        weights = [item.strip() for item in row['weight(g)'].split(',')]
        weight = weights[index]
        actual_weight = portion * float(weight)
        if actual_weight >= 0.0:
            return actual_weight
        retry +=1
    return 'NA'

In [26]:
def calculate_weight(df: pd.DataFrame,ingredient_name: str,portion:str|float, unit:str):
    if portion == 'NA' or unit == 'NA':
        return 'NA'
    return query_df(df,ingredient_name,portion,unit)

In [27]:
calculate_weight(df, 'almond meal', 1.0, 'tablespoon')

5.25

In [28]:
ingredients_enrich = IngredientsEnrich(ingredients=[])
for recipe in recipes.recipes:
    ingredient_enrich = IngredientEnrich(recipe_id=recipe.id, ingredient_unit_portion=[])
    for item in recipe.ingredients:
        ner_name = item.ner_name
        quantity = item.quantity
        portion, unit = parse_quantity(quantity)
        if 'lemon' in ner_name and unit == 'NA':
            unit = 'lemon'
        elif ('stalk' in item.name or 'stalks' in item.name) and unit == 'NA' :
            unit = 'stalk'
        elif 'apple' in item.name and unit == 'NA':
            unit = 'medium' 
        elif ('egg' in item.name or 'eggs' in item.name) and unit == 'NA':
            unit = 'egg'
        elif ('zucchini' in item.name or 'zucchinis' in item.name) and unit == 'NA':
            unit = 'medium'
        elif 'mushroom' in item.name and unit == 'NA':
            unit = 'piece'
        if unit in PLURAL_TO_SINGULAR.keys():
            unit = PLURAL_TO_SINGULAR[unit]
        clean_name = clean_string(ner_name)
        weight = calculate_weight(df,clean_name,portion, unit)
        ingredient_unit_portion = IngredientUnitPortion(name=item.name,
                                                        clean_name=clean_string(ner_name),
                                                        quantity=quantity,
                                                        portion=portion,
                                                        unit=unit,
                                                        weight=weight)
        ingredient_enrich.ingredient_unit_portion.append(ingredient_unit_portion)
    ingredients_enrich.ingredients.append(ingredient_enrich)

In [29]:
import json
with open('../data/generated_recipe_v2/gpt4_1106_enrich_ingredients.json', 'w') as f:
    json.dump(ingredients_enrich.model_dump(mode='json'), f, indent=4, ensure_ascii=False)